In [1]:
!pip install ensemble_boxes

     |████████████████████████████████| 3.3 MB 17.1 MB/s eta 0:00:01
     |████████████████████████████████| 34.5 MB 83.7 MB/s eta 0:00:01


In [9]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO
import os

In [73]:
# ensemble csv files
print(os.getcwd())

submission_files = ['./ensemble_csv/cascade_rcnn_swin-t_1024_KFold_ensemble.csv',
                    './ensemble_csv/cascade_rcnn_swin-t_1024_novalid.csv',
                   './ensemble_csv/sota_0633.csv'] # submission lists
submission_df = [pd.read_csv(file) for file in submission_files]

/opt/ml/level2-object-detection-level2-cv-16/mmdetection


In [74]:
image_ids = submission_df[0]['image_id'].tolist()

In [75]:
# ensemble 할 file의 image 정보를 불러오기 위한 json
annotation = '../../detection/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [76]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
iou_thr = 0.5

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
#     예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [77]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('./ensemble_csv/submission_ensemble.csv')

submission.head()

,PredictionString,image_id
0,0 0.39292398 482.49945 593.68335 575.9327 654....,test/0000.jpg
1,0 0.80307996 639.5073 648.13635 685.08203 836....,test/0001.jpg
2,0 0.96761864 871.9247 464.42337 1022.588 628.9...,test/0002.jpg
3,0 0.3076353 75.00107 96.66156 975.54944 908.48...,test/0003.jpg
4,0 0.975736 185.01 252.175 874.532 777.612 0 0....,test/0004.jpg
